# Discussion-level outcome prediction in Wikipedia's Articles for Deletion debates

In this notebook, we are going to predict the outcome of Wikipedia's Articles for Deletion debates, where editors discuss whether an article should be kept in Wikipedia or deleted. Analysis of Wikipedia Articles for Deletion debates was been originally done by Elijah Mayfield and Alan W Black and published in ["Stance Classification, Outcome Prediction, and Impact Assessment: NLP Tasks for Studying Group Decision-Making"](https://www.aclweb.org/anthology/W19-2108.pdf).


In [1]:
from convokit import Corpus, Speaker, Utterance, VectorClassifier, BoWTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [5]:
afd_corpus = convokit.Corpus(filename=convokit.download("wiki-articles-for-deletion-corpus"))

Here is an example conversation in this corpus. It corresponds to the deletion debate of the **Makoto Uchida** Wikipedia article. After the discussion, it was decided to keep this article. We have `keep` as the `outcome_label`.

In [7]:
afd_corpus.get_conversation('100045461')

Conversation({'obj_type': 'conversation', 'meta': {'article_title': 'Makoto Uchida', 'outcome_id': '300040822', 'outcome_label': 'keep', 'outcome_raw_label': 'keep', 'outcome_decision_maker_id': '200001408', 'outcome_timestamp': 1265155080.0, 'outcome_rationale': "The result was    '''keep'''. '''"}, 'vectors': [], 'tree': None, 'owner': <convokit.model.corpus.Corpus object at 0x7ff09164a880>, 'id': '100045461'})

Let's check what other discussion outcome labels are there in this corpus.

In [8]:
unique_outcomes = set([])
for conv in afd_corpus.iter_conversations():
    unique_outcomes.add(conv.meta['outcome_label'])
    
unique_outcomes, len(unique_outcomes)

({'close',
  'close copyright',
  'close copyright delete',
  'close copyright delete speedy',
  'close delete',
  'close delete keep',
  'close delete keep redirect',
  'close delete keep speedy',
  'close delete keep speedy withdraw',
  'close delete keep withdraw',
  'close delete merge speedy',
  'close delete no-consensus speedy',
  'close delete redirect',
  'close delete redirect speedy',
  'close delete redirect withdraw',
  'close delete speedy',
  'close delete speedy withdraw',
  'close delete withdraw',
  'close keep',
  'close keep redirect',
  'close keep redirect speedy',
  'close keep speedy',
  'close keep speedy withdraw',
  'close keep withdraw',
  'close merge',
  'close merge redirect',
  'close merge redirect speedy',
  'close merge speedy',
  'close move',
  'close move speedy',
  'close no-consensus',
  'close redirect',
  'close redirect speedy',
  'close speedy',
  'close speedy withdraw',
  'close userfy',
  'close withdraw',
  'copyright',
  'copyright delet

There are `174` different outcomes labels! 

Ideally we would like to organize them into two categories -- `keep` and `delete`. We can do so by following conditions used by authors for these labels. To see the original code, check [(1)](https://github.com/emayfield/AFD_Decision_Corpus/blob/e184673699fb2577c8369a1f18adb903d0b57e63/endpoints/data/outcome.py#L221) and then [(2)](https://github.com/emayfield/AFD_Decision_Corpus/blob/e184673699fb2577c8369a1f18adb903d0b57e63/endpoints/data/outcome.py#L59). 

In [9]:
for conv in afd_corpus.iter_conversations():
        outcome_label = conv.meta['outcome_label']
        outcome_label_raw = conv.meta['outcome_raw_label']
    
        if "delete" in outcome_label:
                delete_outcome = 1

        elif "keep" in outcome_label:
                delete_outcome = 0
        
        elif "merge" in outcome_label or \
             "move" in outcome_label or \
             "userfy" in outcome_label or \
             "transwiki" in outcome_label or \
             "incubate" in outcome_label_raw or\
             "redirect" in outcome_label:
                delete_outcome = 1

        elif "withdraw" in outcome_label or \
             "close" in outcome_label or \
             "closing" in outcome_label_raw or \
             "cancel" in outcome_label_raw:
                delete_outcome = 0
        
        elif "speedy" in outcome_label or \
             "copyvio" in outcome_label_raw or \
             "csd" in outcome_label_raw:
                delete_outcome = 1

        else:
                delete_outcome = 0

                
        conv.meta['delete_outcome_binary'] = delete_outcome

Now, a new meta field can tell us whether the articles was decided to be deleted (`delete_outcome_binary=1`) or kept (`delete_outcome_binary=0`).

In [10]:
afd_corpus.get_conversation('100045461')

Conversation({'obj_type': 'conversation', 'meta': {'article_title': 'Makoto Uchida', 'outcome_id': '300040822', 'outcome_label': 'keep', 'outcome_raw_label': 'keep', 'outcome_decision_maker_id': '200001408', 'outcome_timestamp': 1265155080.0, 'outcome_rationale': "The result was    '''keep'''. '''", 'delete_outcome_binary': 0}, 'vectors': [], 'tree': None, 'owner': <convokit.model.corpus.Corpus object at 0x7ff09164a880>, 'id': '100045461'})

Next, we can extract Bag-of-words features for discussions. Recall that in ConvoKit discussion utterances would be concatenated and from there the discussion vector representation is computed.

In [11]:
bow = BoWTransformer(obj_type='conversation',
                      vector_name='convo_bow_vector',
                      vectorizer=CountVectorizer(ngram_range=(1,1), lowercase=True, min_df=1))

afd_corpus = bow.fit_transform(afd_corpus)

Thus, every discussion has an associated BoW vector. 

In [12]:
c = afd_corpus.random_conversation()
c.get_vector('convo_bow_vector')

<1x1117438 sparse matrix of type '<class 'numpy.int64'>'
	with 155 stored elements in Compressed Sparse Row format>

To predict the discussion outcome we are going to use ConvoKit's VectorClassifier. Let's chose the classification model to be liblinear logistic regression as done by the authors. 

In [13]:
bow_classifier = VectorClassifier(obj_type="conversation", 
                                  vector_name="convo_bow_vector",
                                  labeller=lambda conv: conv.meta['delete_outcome_binary'],
                                  clf=LogisticRegression(solver='liblinear'))

Evaluate with cross-validation.

In [14]:
results_bow = bow_classifier.evaluate_with_cv(afd_corpus, cv=KFold(n_splits=5))
results_bow

Running a cross-validated evaluation...Done.


array([0.86070015, 0.86163784, 0.85965826, 0.86407147, 0.86193558])

In [17]:
np.average(results_bow)

0.8616006616145521